In [2125]:
# https://preppindata.blogspot.com/2021/06/2021-week-25-worst-pokemon.html

import pandas as pd
import numpy as np
from datetime import date, timedelta

### Input the data

In [2126]:
df_gen1 = pd.read_excel('data\PD 2021 Wk 25 Input.xlsx', sheet_name='Gen 1')
df_evo_gp = pd.read_excel('data\PD 2021 Wk 25 Input.xlsx', sheet_name='Evolution Group')
df_evo = pd.read_excel('data\PD 2021 Wk 25 Input.xlsx', sheet_name='Evolutions')
df_mega_evo = pd.read_excel('data\PD 2021 Wk 25 Input.xlsx', sheet_name='Mega Evolutions')
df_alolan = pd.read_excel('data\PD 2021 Wk 25 Input.xlsx', sheet_name='Alolan')
df_galarian = pd.read_excel('data\PD 2021 Wk 25 Input.xlsx', sheet_name='Galarian')
df_gigantamax = pd.read_excel('data\PD 2021 Wk 25 Input.xlsx', sheet_name='Gigantamax')
df_unattainable = pd.read_excel('data\PD 2021 Wk 25 Input.xlsx', sheet_name='Unattainable in Sword & Shield')
df_anime = pd.read_excel('data\PD 2021 Wk 25 Input.xlsx', sheet_name='Anime Appearances')

### Clean up the list of Gen 1 Pokémon so we have 1 row per Pokémon

In [2127]:
df_gen1 = df_gen1[['#','Name']]
df_gen1 = df_gen1[df_gen1['#'].notnull()]
df_gen1['#'] = df_gen1['#'].astype(int)
df_gen1

len(set(df_gen1['Name']))

151

### Clean up the Evolution Group input so that we can join it to the Gen 1 list 
- Filter out Starter and Legendary Pokémon

In [2128]:
df_evo_gp_clean = df_evo_gp.copy()
#df_evo_gp_clean = df_evo_gp[(df_evo_gp_clean['Starter?']==0) & (df_evo_gp_clean['Legendary?'] == 0)]
#df_evo_gp_clean.drop(['Starter?','Legendary?'],axis=1,inplace=True)

df_evo_gp_clean['#'] = df_evo_gp_clean['#'].astype(int)
df_evo_gp_clean.drop_duplicates(inplace=True)

len(set(df_evo_gp_clean['#']))

151

In [2129]:
df_gen1_w_gp = pd.merge(df_gen1,df_evo_gp_clean, on = '#', how = 'inner')


df_gen1_w_gp = df_gen1_w_gp[(df_gen1_w_gp['Starter?'] == 0) & (df_gen1_w_gp['Legendary?'] == 0)]
df_LK_gen1_w_gp = df_gen1_w_gp.copy()
len(set(df_gen1_w_gp['Name']))

131

In [2130]:
df_gen1_w_gp

,#,Name,Evolution Group,Starter?,Legendary?
9,10,Caterpie,Caterpie,0,0
10,11,Metapod,Caterpie,0,0
11,12,Butterfree,Caterpie,0,0
12,13,Weedle,Weedle,0,0
13,14,Kakuna,Weedle,0,0
...,...,...,...,...,...
141,142,Aerodactyl,Aerodactyl,0,0
142,143,Snorlax,Snorlax,0,0
146,147,Dratini,Dratini,0,0
147,148,Dragonair,Dratini,0,0


### Using the Evolutions input, exclude any Pokémon that evolves from a Pokémon that is not part of Gen 1 or can evolve into a Pokémon outside of Gen 1

In [2131]:
df_evo_filtered = df_evo[(df_evo['Evolving to'].isin(df_gen1_w_gp['Name']))
                         & (df_evo['Evolving from'].isin(df_gen1_w_gp['Name']))]
df_evo_filtered

,Evolving from,Evolving to,Level,Condition,Evolution Type
6,Caterpie,Metapod,7.0,NaN,Level
7,Metapod,Butterfree,10.0,NaN,Level
8,Weedle,Kakuna,7.0,NaN,Level
9,Kakuna,Beedrill,10.0,NaN,Level
10,Pidgey,Pidgeotto,18.0,NaN,Level
...,...,...,...,...,...
278,Staryu,Starmie,NaN,Water Stone,Stone
308,Kadabra,Alakazam,NaN,NaN,Trade
309,Machoke,Machamp,NaN,NaN,Trade
310,Graveler,Golem,NaN,NaN,Trade


In [2132]:
len(set(df_evo_filtered['Evolving from']))

62

In [2133]:
len(set(df_evo_filtered['Evolving to']))

62

In [2134]:
df_inc_ls_evo =  set(df_evo_filtered['Evolving from']) | set(df_evo_filtered['Evolving to'])
df_inc_ls_evo = list(df_inc_ls_evo)
len(df_inc_ls_evo)

111

In [2135]:
df_gen1_w_gp = df_gen1_w_gp[df_gen1_w_gp['Name'].isin(df_inc_ls_evo)]
df_gen1_w_gp

,#,Name,Evolution Group,Starter?,Legendary?
9,10,Caterpie,Caterpie,0,0
10,11,Metapod,Caterpie,0,0
11,12,Butterfree,Caterpie,0,0
12,13,Weedle,Weedle,0,0
13,14,Kakuna,Weedle,0,0
...,...,...,...,...,...
139,140,Kabuto,Kabuto,0,0
140,141,Kabutops,Kabuto,0,0
146,147,Dratini,Dratini,0,0
147,148,Dragonair,Dratini,0,0


In [2136]:
len(set(df_gen1_w_gp['Name'].unique()))

111

### Exclude any Pokémon with a mega evolution, Alolan, Galarian or Gigantamax form

In [2137]:
df_exc_ls = pd.concat([df_mega_evo,df_alolan,df_galarian,df_gigantamax])

#clean prefix and suffix which doesn't exist in the pokemon like
df_exc_ls['Name'] = df_exc_ls['Name'].str.replace('(Mega|Alolan|Galarian|Gigantamax) ','', regex=True)
df_exc_ls['Name'] = df_exc_ls['Name'].str.replace(' (X|Y)$','', regex=True)
df_exc_ls = df_exc_ls['Name'].drop_duplicates()

In [2138]:
# find any pokemon that evolves into a mega / gigantamax
id_lookup = df_gen1_w_gp[['#','Name']]
df_exc_ls = pd.merge(df_exc_ls,id_lookup, on = 'Name', how = 'inner')

df_exc_ls = pd.merge(df_exc_ls,df_evo_gp_clean, on = '#', how = 'inner')
del df_exc_ls['#']
df_exc_ls = pd.merge(df_exc_ls,df_evo_gp_clean, on = 'Evolution Group', how = 'inner')
df_exc_ls

,Name,Evolution Group,Starter?_x,Legendary?_x,#,Starter?_y,Legendary?_y
0,Beedrill,Weedle,0,0,13,0,0
1,Beedrill,Weedle,0,0,14,0,0
2,Beedrill,Weedle,0,0,15,0,0
3,Pidgeot,Pidgey,0,0,16,0,0
4,Pidgeot,Pidgey,0,0,17,0,0
...,...,...,...,...,...,...,...
64,Machamp,Machop,0,0,66,0,0
65,Machamp,Machop,0,0,67,0,0
66,Machamp,Machop,0,0,68,0,0
67,Kingler,Krabby,0,0,98,0,0


In [2139]:
# not in special evolution
df_gen1_w_gp = df_gen1_w_gp[~df_gen1_w_gp['#'].isin(df_exc_ls['#'])]
df_gen1_w_gp

,#,Name,Evolution Group,Starter?,Legendary?
20,21,Spearow,Spearow,0,0
21,22,Fearow,Spearow,0,0
22,23,Ekans,Ekans,0,0
23,24,Arbok,Ekans,0,0
28,29,Nidoran♀,Nidoran♀,0,0
...,...,...,...,...,...
139,140,Kabuto,Kabuto,0,0
140,141,Kabutops,Kabuto,0,0
146,147,Dratini,Dratini,0,0
147,148,Dragonair,Dratini,0,0


In [2140]:
len(set(df_gen1_w_gp['Name'].unique()))

64

### It's not possible to catch certain Pokémon in the most recent games. These are the only ones we will consider from this point on

In [2141]:
df_unattainable['Name']

0         Weedle
1         Pidgey
2        Rattata
3        Spearow
4          Ekans
5          Paras
6        Venonat
7         Mankey
8     Bellsprout
9        Geodude
10         Doduo
11          Seel
12        Grimer
13       Drowzee
14       Voltorb
Name: Name, dtype: object

In [2142]:
df_gen1_w_gp = df_gen1_w_gp[df_gen1_w_gp['Name'].isin(df_unattainable['Name'].to_list())]
df_gen1_w_gp['Name'].unique()

array(['Spearow', 'Ekans', 'Paras', 'Venonat', 'Mankey', 'Bellsprout',
       'Doduo', 'Seel', 'Drowzee', 'Voltorb'], dtype=object)

In [2143]:
df_gen1_w_gp

,#,Name,Evolution Group,Starter?,Legendary?
20,21,Spearow,Spearow,0,0
22,23,Ekans,Ekans,0,0
45,46,Paras,Paras,0,0
47,48,Venonat,Venonat,0,0
55,56,Mankey,Mankey,0,0
68,69,Bellsprout,Bellsprout,0,0
83,84,Doduo,Doduo,0,0
85,86,Seel,Seel,0,0
95,96,Drowzee,Drowzee,0,0
99,100,Voltorb,Voltorb,0,0


In [2144]:
df_anime

,Episode,Pokemon
0,EP001,Pikachu
1,EP001,Nidorino
2,EP001,Onix
3,EP001,Gengar
4,EP001,Spearow
...,...,...
1828,EP113,Marill
1829,EP113,Blastoise
1830,EP113,Tentacruel
1831,EP113,Lapras


### We're left with 10 evolution groups. Rank them in ascending order of how many times they've appeared in the anime to see who the worst Pokémon is!

In [2145]:
#check if we have 10 evo group
df_gen1_w_gp['Name'].unique()
df_gen1_w_gp

,#,Name,Evolution Group,Starter?,Legendary?
20,21,Spearow,Spearow,0,0
22,23,Ekans,Ekans,0,0
45,46,Paras,Paras,0,0
47,48,Venonat,Venonat,0,0
55,56,Mankey,Mankey,0,0
68,69,Bellsprout,Bellsprout,0,0
83,84,Doduo,Doduo,0,0
85,86,Seel,Seel,0,0
95,96,Drowzee,Drowzee,0,0
99,100,Voltorb,Voltorb,0,0


In [2146]:
df_anime_w_gp = df_anime.merge(df_LK_gen1_w_gp, left_on='Pokemon', right_on='Name',how='inner')
df_anime_w_gp = df_anime_w_gp[['Evolution Group','Episode']].drop_duplicates()
df_anime_w_gp


,Evolution Group,Episode
0,Nidoran♂,EP001
1,Nidoran♂,EP033
2,Nidoran♂,EP035
3,Nidoran♂,EP047
4,Nidoran♂,EP057
...,...,...
1364,Horsea,EP098
1365,Horsea,EP111
1367,Nidoran♀,EP093
1368,Nidoran♀,EP098


In [2152]:
df_appearance_target = df_anime_w_gp[df_anime_w_gp['Evolution Group'].isin(list(df_gen1_w_gp['Evolution Group']))]
df_appearance_target

,Evolution Group,Episode
43,Spearow,EP001
44,Spearow,EP002
45,Spearow,EP011
46,Spearow,EP030
47,Spearow,EP031
...,...,...
1267,Paras,EP116
1331,Seel,EP099
1352,Doduo,EP065
1353,Doduo,EP069


In [2150]:
df_appearance_target = df_appearance_target.groupby(['Evolution Group'])['Episode'].count().reset_index()
df_appearance_target

df_appearance_target['The Worse pokemon'] = df_appearance_target['Episode'].rank().astype(int)
df_appearance_target.rename(columns={'Episode': 'Appearances'},inplace=True)
df_appearance_target = df_appearance_target[['The Worse pokemon','Evolution Group','Appearances']].sort_values(by='The Worse pokemon')
df_appearance_target

,The Worse pokemon,Evolution Group,Appearances
2,1,Drowzee,3
6,2,Seel,8
5,3,Paras,13
1,5,Doduo,15
7,5,Spearow,15
8,5,Venonat,15
4,7,Mankey,16
9,7,Voltorb,16
0,9,Bellsprout,31
3,10,Ekans,50


### Output the data

In [2151]:
df_appearance_target.to_csv(r'output/2021-week25-output.csv')